<a href="https://colab.research.google.com/github/herysedra/covid19-mankaiza-clone/blob/andrana/scripts/paper/simple_blocks/sir_mod_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import matplotlib
import pickle

import pymc3 as pm
import theano.tensor as tt
import theano
import datetime
import time

In [21]:
NT = 2e6
N10 = 1e4
N1 = 1e3

ST = NT - 54
S10 = N10 - 54
S1 = N1 - 54

ST3 = NT - 54
S103 = N10 - 54
S13 = N1 - 54

ItotT = 54
IvT = 4
IT = 54 # mbola tsy nisy sitrana hatreo

Itot10 = 54
Iv10 = 4
I10 = 54 

Itot1 = 54
Iv1 = 4
I1 = 54 

ItotT3 = 54
IvT3 = 4
IT3 = 54 

Itot103 = 54
Iv103 = 4
I103 = 54 

Itot13 = 54
Iv13 = 4
I13 = 54 

lc = 0.12
mc = 0.15

l3 = 0.45
m3 = 0.15

i = 1

for el in range(15):
  i =  i + 1
  IvT = (lc/NT)*ST*IT
  ST = ST - (lc/NT)*IT
  IT = IT + IvT - mc*IT
  ItotT = ItotT + IvT
  Iv10 = (lc/N10)*S10*I10
  S10 = S10 - (lc/N10)*I10
  I10 = I10 + Iv10 - mc*I10
  Itot10 = Itot10 + Iv10 
  Iv1 = (lc/N1)*S1*I1
  S1 = S1 - (lc/N1)*I1
  I1 = I1 + Iv1 - mc*I1
  Itot1 = Itot1 + Iv1 
  IvT3 = (l3/NT)*ST3*IT3
  ST3 = ST3 - (l3/NT)*IT3
  IT3 = IT3 + IvT3 - m3*IT3
  ItotT3 = ItotT3 + IvT3
  Iv103 = (l3/N10)*S103*I103
  S103 = S103 - (l3/N10)*I103
  I103 = I103 + Iv103 - m3*I103
  Itot103 = Itot103 + Iv103 
  Iv13 = (l3/N1)*S13*I13
  S13 = S13 - (l3/N1)*I13
  I13 = I13 + Iv13 - m3*I13
  Itot13 = Itot13 + Iv13
  print(i, '/4 &', round(ItotT, 2), '&', round(IvT, 2), '&', round(Itot10, 2), '&', round(Iv10, 2), '&', round(Itot1, 2), '&', round(Iv1, 2), '&', round(ItotT3, 2), '&', round(IvT3, 2), '&', round(Itot103, 2), '&', round(Iv103, 2), '&', round(Itot13, 2), '&', round(Iv13, 2))

2 /4 & 60.48 & 6.48 & 60.45 & 6.45 & 60.13 & 6.13 & 78.3 & 24.3 & 78.17 & 24.17 & 76.99 & 22.99
3 /4 & 66.77 & 6.29 & 66.69 & 6.25 & 66.04 & 5.91 & 109.89 & 31.59 & 109.53 & 31.36 & 106.31 & 29.32
4 /4 & 72.86 & 6.1 & 72.75 & 6.06 & 71.73 & 5.69 & 150.95 & 41.07 & 150.22 & 40.69 & 143.72 & 37.41
5 /4 & 78.78 & 5.91 & 78.62 & 5.87 & 77.21 & 5.48 & 204.34 & 53.38 & 203.02 & 52.8 & 191.44 & 47.72
6 /4 & 84.51 & 5.74 & 84.31 & 5.69 & 82.49 & 5.28 & 273.74 & 69.4 & 271.54 & 68.51 & 252.31 & 60.87
7 /4 & 90.08 & 5.56 & 89.83 & 5.52 & 87.58 & 5.09 & 363.95 & 90.22 & 360.44 & 88.9 & 329.95 & 77.64
8 /4 & 95.47 & 5.4 & 95.17 & 5.35 & 92.49 & 4.9 & 481.24 & 117.28 & 475.79 & 115.36 & 428.98 & 99.03
9 /4 & 100.71 & 5.24 & 100.36 & 5.18 & 97.21 & 4.73 & 633.7 & 152.46 & 625.47 & 149.68 & 555.29 & 126.31
10 /4 & 105.79 & 5.08 & 105.38 & 5.02 & 101.77 & 4.55 & 831.9 & 198.2 & 819.7 & 194.22 & 716.37 & 161.09
11 /4 & 110.71 & 4.93 & 110.25 & 4.87 & 106.15 & 4.39 & 1089.56 & 257.66 & 1071.71 & 252.01 

In [0]:
"""
λ = tt.iscalar("λ")
μ = tt.iscalar("μ")
S_t = tt.matrix("S_t")
I_t = tt.matrix("I_t")
N = tt.iscalar("N")
S_begin = tt.matrix("S_begin")
I_begin = tt.matrix("I_begin")
I_all = tt.matrix
andro = tt.iscalar("andro")

s0 = tt.matrix()
i0 = tt.matrix()
ni0 = tt.matrix()
"""

def SIR_model(λ, μ, S_begin, I_begin, N):
    new_I_0 = tt.zeros_like(I_begin)
    def next_day(λ, S_t, I_t, _):
        new_I_t = λ/N*I_t*S_t
        S_t = S_t - new_I_t
        I_t = I_t + new_I_t - μ * I_t
        return S_t, I_t, new_I_t
    outputs , _  = theano.scan(fn=next_day, sequences=[λ],
                               n_steps = andro, 
                               outputs_info=[S_begin, I_begin, new_I_0])
    S_all, I_all, new_I_all = outputs
    return S_all, I_all, new_I_all 

N_tana = 2000000
I_begin = 54
S_begin = N_tana - I_begin

S_past, I_past, new_I_past = SIR_model(λ=0.45*tt.ones(14), μ=0.15, 
                                               S_begin=S_begin, I_begin=I_begin,
                                               N=N_tana)
print(new_I_past)

AttributeError: ignored

In [0]:
def SIR_model(λ, μ, S_begin, I_begin, N):
    new_I_0 = tt.zeros_like(I_begin)
    def next_day(λ, S_t, I_t, _):
        new_I_t = λ/N*I_t*S_t
        S_t = S_t - new_I_t
        I_t = I_t + new_I_t - μ * I_t
        return S_t, I_t, new_I_t
    outputs , _  = theano.scan(fn=next_day, sequences=[λ], 
                               outputs_info=[S_begin, I_begin, new_I_0])
    S_all, I_all, new_I_all = outputs
    return S_all, I_all, new_I_all

#  SIR_mod_funct = theano.function(
#     inputs = λ, μ, S_begin, I_begin, N
#     outputs = S_past, I_past, new_I_past
    )
with pm.Model() as model:
    # true cases at begin of loaded data but we do not know the real number
    I_begin = pm.HalfCauchy('I_begin', beta=100)

    # fraction of people that are newly infected each day
    λ = pm.Lognormal("λ", mu=np.log(0.4), sigma=0.5)

    # fraction of people that recover each day, recovery rate mu
    μ = pm.Lognormal('μ', mu=np.log(1/8), sigma=0.2)

    # delay in days between contracting the disease and being recorded
    delay = pm.Lognormal("delay", mu=np.log(8), sigma=0.2)

    # prior of the error of observed cases
    σ_obs = pm.HalfCauchy("σ_obs", beta=10)

    N_germany = 83e6

   # -------------------------------------------------------------------------- #
    # training the model with loaded data
    # -------------------------------------------------------------------------- #

    S_begin = N_germany - I_begin
    S_past, I_past, new_I_past = SIR_model(λ=λ * tt.ones(num_days-1), μ=μ, 
                                               S_begin=S_begin, I_begin=I_begin,
                                               N=N_germany)
    new_cases_obs = np.diff(cases_obs)
    new_cases_inferred = delay_cases(new_I_past, len_new_I_t=num_days - 1, 
                                     len_new_cases_obs=len(new_cases_obs), 
                                     delay=delay, delay_arr=diff_data_sim)

    # Approximates Poisson
    # calculate the likelihood of the model:
    # observed cases are distributed following studentT around the model
    pm.StudentT(
        "obs",
        nu=4,
        mu=new_cases_inferred,
        sigma=(new_cases_inferred)**0.5 * σ_obs,
        observed=new_cases_obs)  
    
    S_past = pm.Deterministic('S_past', S_past)
    I_past = pm.Deterministic('I_past', I_past)
    new_I_past = pm.Deterministic('new_I_past', new_I_past)
    new_cases_past = pm.Deterministic('new_cases_past', new_cases_inferred)
    
    """"
    Mahasedra: 
    pymc3.model.Deterministic(name, var, model=None)¶
    Create a named deterministic variable
    Parameters: name string ; var theano variables
    Returns : var, with name attribute
    
    In the case of the simple SIR model, the variables S, new_I and 
    new_(reported)cases are determined from the SIR model that why they are 
    deterministic.
    """"

    # -------------------------------------------------------------------------- #
    # prediction, start with no changes in policy
    # -------------------------------------------------------------------------- #

    S_begin = S_past[-1]
    I_begin = I_past[-1]
    forecast_no_change = SIR_model(λ=λ*tt.ones(num_days_to_predict), μ=μ, 
                        S_begin=S_begin, I_begin=I_begin, N=N_germany)
    S_no_change, I_no_change, new_I_no_change = forecast_no_change

    #saves the variables for later retrieval
    pm.Deterministic('S_no_change', S_no_change)
    pm.Deterministic('I_no_change', I_no_change)
    pm.Deterministic('new_I_no_change', new_I_no_change)

    new_cases_inferred = delay_cases(tt.concatenate([new_I_past[-diff_data_sim:], new_I_no_change]), 
                                     len_new_I_t=diff_data_sim + num_days_to_predict, 
                                     len_new_cases_obs=num_days_to_predict, 
                                     delay=delay, delay_arr=diff_data_sim)
    pm.Deterministic('new_cases_no_change', new_cases_inferred)



In [0]:
   # -------------------------------------------------------------------------- #
    # social distancing, m reduced by about 50 percent
    # -------------------------------------------------------------------------- #
    #For all following predictions:
    length_transient = 7  # days


    # λ is decreased by 50%
    reduc_factor_mild = 0.5
    days_offset = 0  # start the decrease in spreading rate after this

    time_arr = np.arange(num_days_to_predict)

    # change in m along time
    λ_correction = tt.clip((time_arr - days_offset) / length_transient, 0, 1)
    λ_t_soc_dist= λ * (1 - λ_correction * reduc_factor_mild) 

    S_begin = S_past[-1]
    I_begin = I_past[-1]
    forecast_soc_dist = SIR_model(λ=λ_t_soc_dist, μ=μ, 
                        S_begin=S_begin, I_begin=I_begin, 
                        N=N_germany)
    S_soc_dist, I_soc_dist, new_I_soc_dist = forecast_soc_dist
    pm.Deterministic('S_soc_dist', S_soc_dist)
    pm.Deterministic('I_soc_dist', I_soc_dist)
    pm.Deterministic('new_I_soc_dist', new_I_soc_dist)

    new_cases_inferred = delay_cases(tt.concatenate([new_I_past[-diff_data_sim:], new_I_soc_dist]), 
                                    len_new_I_t=diff_data_sim + num_days_to_predict, 
                                    len_new_cases_obs=num_days_to_predict, 
                                    delay=delay, delay_arr=diff_data_sim)
    pm.Deterministic('new_cases_soc_dist', new_cases_inferred)

In [0]:
  # -------------------------------------------------------------------------- #
    # isolation, almost no new infections besides baseline after transient phase
    # -------------------------------------------------------------------------- #

    # λ is decreased by 90%
    reduc_factor_strong = 0.9
    days_offset = 0  # start the decrease in spreading rate after this

    # spreading of people who transmit although they are isolated
    time_arr = np.arange(num_days_to_predict)

    # change in λ along time
    λ_correction = tt.clip((time_arr - days_offset) / length_transient, 0, 1)
    λ_t_isol= λ * (1 - λ_correction * reduc_factor_strong)

    S_begin = S_past[-1]
    I_begin = I_past[-1]
    forecast_isol = SIR_model(λ=λ_t_isol , μ=μ, 
                              S_begin=S_begin, I_begin=I_begin, 
                              N=N_germany)
    S_isol, I_isol, new_I_isol = forecast_isol

    pm.Deterministic('S_isol', S_isol)
    pm.Deterministic('I_isol', I_isol)  
    pm.Deterministic('new_I_isol', new_I_isol)

    new_cases_inferred = delay_cases(tt.concatenate([new_I_past[-diff_data_sim:], new_I_isol]), 
                                len_new_I_t=diff_data_sim + num_days_to_predict, 
                                len_new_cases_obs=num_days_to_predict, 
                                delay=delay, delay_arr=diff_data_sim)
    pm.Deterministic('new_cases_isol', new_cases_inferred)
